In [31]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [32]:
img_size=7936
np.random.seed(1234567)
#loading data and fitting follows:
X_train=np.loadtxt('training_data/Xtrain_ush_short.csv',delimiter=',')
Y_train=np.loadtxt('training_data/Ytrain_ush_short.csv',delimiter=',')
#Xtrain=Xtrain[:2000]
X_train=np.transpose(X_train)
print(X_train.shape)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_train, Y_train, test_size=0.33)

(3447, 7936)


In [46]:
#defining a sampling function: returns a random sample from a mean and variance
#that is input to it
print(Xtrain.shape,Xtest.shape)
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = keras.backend.random_normal(shape=keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + keras.backend.exp(log_variance/2) * epsilon
    return random_sample

(2309, 7936, 1) (1138, 7936)


In [47]:
#model defined here
inp = keras.layers.Input(shape=(img_size, 1), name="encoder_input")
#leaky reLU has been added as a separate layer than define as an activation
x1 = keras.layers.Conv1D(filters=1, kernel_size=5, padding="same", strides=2,name="encoder_conv_1")(inp)
x1 = keras.layers.BatchNormalization(name="encoder_norm_1")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(x1)

x1 = keras.layers.Conv1D(filters=16, kernel_size=5, padding="same", strides=2, name="encoder_conv_2")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_2")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(x1)

x1 = keras.layers.Conv1D(filters=32, kernel_size=5, padding="same", strides=2, name="encoder_conv_3")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_3")(x1)
bp_lay_1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(x1)
#flatten the layers in encoder
shape_before_flatten = keras.backend.int_shape(bp_lay_1)[1:]
x2 = keras.layers.Flatten(name="flat_1")(bp_lay_1)

latent_space_dim = 4
#declare a mean and variance for the distribution
encoder_mu = keras.layers.Dense(units=latent_space_dim, name="encoder_mu")(x2)
encoder_log_variance = keras.layers.Dense(units=latent_space_dim, name="encoder_log_variance")(x2)
encoder_op = keras.layers.Lambda(sampling, name="encoder_output")([encoder_mu, encoder_log_variance])

encoder = keras.models.Model(inp, encoder_op, name="encoder_model")
encoder.summary()


Model: "encoder_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 7936, 1)]    0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv1D)         (None, 3968, 1)      6           encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_norm_1 (BatchNormalizat (None, 3968, 1)      4           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_leakyrelu_1 (LeakyReLU) (None, 3968, 1)      0           encoder_norm_1[0][0]             
______________________________________________________________________________________

In [48]:
#decoder_input = keras.layers.Input(shape=(shape_before_flatten), name="decoder_input")
decoder_input = keras.layers.Input(shape=(latent_space_dim), name="decoder_input")
x3 = keras.layers.Dense(units=np.prod(shape_before_flatten), name="decoder_dense_1")(decoder_input)
x3 = keras.layers.Reshape(target_shape=shape_before_flatten)(x3)

x3 = keras.layers.Conv1DTranspose(filters=32, kernel_size=5, padding="same", strides=2, name="decoder_conv_tran_1")(x3)
x3 = keras.layers.BatchNormalization(name="decoder_norm_1")(x3)
x3 = keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(x3)

x3 = keras.layers.Conv1DTranspose(filters=16, kernel_size=5, padding="same", strides=2, name="decoder_conv_tran_2")(x3)
x3 = keras.layers.BatchNormalization(name="decoder_norm_2")(x3)
x3 = keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(x3)

decoder_output = keras.layers.Conv1DTranspose(filters=1, kernel_size=5, padding="same", strides=2,activation='sigmoid', name="decoder_conv_tran_4")(x3)
#decoder_output = keras.layers.LeakyReLU(name="decoder_output")(x3)

decoder = keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()


Model: "decoder_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 4)]               0         
_________________________________________________________________
decoder_dense_1 (Dense)      (None, 31744)             158720    
_________________________________________________________________
reshape_6 (Reshape)          (None, 992, 32)           0         
_________________________________________________________________
decoder_conv_tran_1 (Conv1DT (None, 1984, 32)          5152      
_________________________________________________________________
decoder_norm_1 (BatchNormali (None, 1984, 32)          128       
_________________________________________________________________
decoder_leakyrelu_1 (LeakyRe (None, 1984, 32)          0         
_________________________________________________________________
decoder_conv_tran_2 (Conv1DT (None, 3968, 16)        

In [49]:
from keras import backend as K
def loss_func(encoder_mu, encoder_log_variance):
    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = keras.backend.mean(keras.backend.square(y_true-y_predict), axis=[1, 2])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * keras.backend.sum(1.0 + encoder_log_variance - keras.backend.square(encoder_mu) - keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * keras.backend.sum(1.0 + encoder_log_variance - keras.backend.square(encoder_mu) - keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss



In [50]:
#model for the vae
vae_input = keras.layers.Input(shape=(img_size,1), name="VAE_input")
vae_encoder_output = encoder(vae_input)
vae_decoder_output = decoder(vae_encoder_output)
vae = keras.models.Model(vae_input, vae_decoder_output, name="VAE")

vae.summary()
#vae.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=loss_func(encoder_mu, encoder_log_variance))



Model: "VAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
VAE_input (InputLayer)       [(None, 7936, 1)]         0         
_________________________________________________________________
encoder_model (Functional)   (None, 4)                 256850    
_________________________________________________________________
decoder_model (Functional)   (None, 7936, 1)           166721    
Total params: 423,571
Trainable params: 423,377
Non-trainable params: 194
_________________________________________________________________


In [51]:
#Xtrain_N=[]
#[Xtrain_N.append(Xtrain[i]/np.abs(Xtrain[i][np.argmin(Xtrain[i])])) for i in range(0,3231)]
#Xtrain=np.array(Xtrain_N).reshape(3231,12000,1)
#arr=np.arange(0,4500,1)
#np.random.shuffle(arr)
#Xtrain=np.array([Xtrain[i] for i in arr])
#Xtrain=np.array(Xtrain).reshape(4500,2000,1)
#Xtrain=Xtrain[0:4500]
#print(Xtrain.shape)
Xtrain=[preprocessing.normalize(Xtrain[i]) for i in range(0,len(Xtrain))]
Xtrain=np.array(Xtrain).reshape(2309, 7936, 1)
print(Xtrain.shape)

(2309, 7936, 1)


In [52]:

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history=vae.fit(Xtrain, Xtrain, epochs=30, batch_size=64 ,verbose=1, validation_split=0.2)

Epoch 1/30
29/29 [==============================] - 56s 2s/step - loss: 5759.8721 - val_loss: 5731.1870
Epoch 2/30
29/29 [==============================] - 52s 2s/step - loss: 5649.2168 - val_loss: 5650.8779
Epoch 3/30
29/29 [==============================] - 52s 2s/step - loss: 5566.2344 - val_loss: 5564.6528
Epoch 4/30
29/29 [==============================] - 50s 2s/step - loss: 5486.0527 - val_loss: 5480.1572
Epoch 5/30
29/29 [==============================] - 51s 2s/step - loss: 5416.5684 - val_loss: 5405.3818
Epoch 6/30
29/29 [==============================] - 51s 2s/step - loss: 5340.0444 - val_loss: 5332.9409
Epoch 7/30
29/29 [==============================] - 51s 2s/step - loss: 5253.9062 - val_loss: 5257.4712
Epoch 8/30
29/29 [==============================] - 49s 2s/step - loss: 5183.1685 - val_loss: 5183.1533
Epoch 9/30
29/29 [==============================] - 43s 1s/step - loss: 5134.2026 - val_loss: 5126.6172
Epoch 10/30
29/29 [==============================] - 43s 1s/step

In [53]:
vae.save_weights('autoencoder_raw_7936.h5')
encoder.save_weights('encoding_raw_7936.h5')
decoder.save_weights('decoding_simpler_raw_7936.h5')